# Controlling system Audio using your Fingers in Python and Computer vision

In [4]:
import cv2
import numpy as np
from math import hypot
import mediapipe as mp
from comtypes import CLSCTX_ALL
import matplotlib.pyplot as plt
from ctypes import cast, POINTER
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [5]:
mpHands = mp.solutions.hands # detects hands/fingers

hands = mpHands.Hands() # complete the initilization configuration for hands

mpDraw = mp.solutions.drawing_utils

In [8]:
# To access speaker through the library pycaw
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

volume = cast(interface, POINTER(IAudioEndpointVolume))

volbar = 400
volper = 0

volMin, volMax = volume.GetVolumeRange()[:2]
volMin, volMax

(-65.25, 0.0)

In [19]:
cap = cv2.VideoCapture(0) # Checks for camera

In [ ]:
while True:
    success, img = cap.read() # if camera works captur image
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # plt.imshow(imgRGB)
    
    # collection of gesture information
    results = hands.process(imgRGB)
    
    # empty list
    lmList = [] 

    # List of all hands detected
    if results.multi_hand_landmarks: 
        
        # By accessing tje list, we can get the informationn of each hands corrosponding flag         
        for handLandmark in results.multi_hand_landmarks:
            # adding counter and returning it
            for id, lm in enumerate(handLandmark.landmark):
                # Get finger joint points
                h,w,_ = img.shape
                cx, cy = int(lm.x*w), int(lm.y*h)
                # adding to the empty list 'lmList'
                lmList.append([id,cx,cy])
            mpDraw.draw_landmarks(img, handLandmark, mpHands.HAND_CONNECTIONS)
        
    if lmList != []:
        # Getting the value at a point
                    # x          # y
        x1, y1 = lmList[4][1], lmList[4][2] # Thumb
        x2, y2 = lmList[8][1], lmList[8][2] # Index
        # creating circle at the tips of thumb and index finger
        cv2.circle(img, (x1,y1),13,(255,0,0),cv2.FILLED) # image # finger # radius # rgb
        cv2.circle(img, (x2,y2),13,(255,0,0),cv2.FILLED) # image # finger # radius # rgb
        cv2.line(img,(x1,y1),(x2,y2),(255,0,0),3) # create a line b/w tips of index finger and
        
        length = hypot(x2-x1,y2-y1) # distance between tips using hypotenuse
        
        # from numpy we find our length by converting hand range in terms of volume range i.e. b/w -65.25, 0.0
        
        vol = np.interp(length, [30,350],[volMin, volMax])
        volbar = np.interp(length, [30,350], [400, 150])
        volper = np.interp(length, [30,350], [0, 100])
        
        # print(vol, int(length))
        volume.SetMasterVolumeLevel(vol, None)
        
        # Hand range 30 - 350
        # Volume range -63.5 - 0
        # creating volume bar for volume level.
        cv2.rectangle(img, (50,150), (85,400),(0,0,255),4) # vid, initial position, ending position
        cv2.rectangle(img, (50, int(volbar)),(85,400),(0,0,255),cv2.FILLED)
        cv2.putText(img, f"{int(volper)}%", (10,40), cv2.FONT_ITALIC, 1, (0,255,98), 3)
        # tell the volume percent, location, font of text, length, rgb color, thickness
    cv2.imshow("Image", img) # show the video
    # by using space bar delay will stop
    if cv2.waitKey(1) & 0xff==ord(' '):
        break
        
        
# stop video capture
cap.release()
# close window
cv2.destroyAllWindows()    